In [ ]:
# 해당 셀을 실행한 후에 반드시 "런타임 다시시작"을 해주세요
!pip3 install kfp google-cloud-aiplatform --upgrade -q --user

In [ ]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component, ClassificationMetrics, Metrics)
from kfp.v2.google import experimental

from google.cloud import aiplatform as vertex_ai

## GCP 연결

In [ ]:
# gcp 연결
from google.colab import auth as google_auth

google_auth.authenticate_user()

# 환경변수 설정
아래 항목들을 본인 환경에 맞게 수정해주세요.

* PROJECT_ID = <프로젝트 ID>
* REGION = <리전>
* BUCKET_NAME = <bucket 이름>
* USER = <user 이름>

In [ ]:
# GCP 설정
PROJECT_ID = 'gold-doodad-326906'
REGION = "us-central1"
USER = "JeongMin-Do"
BUCKET = "mlops-1234"
PIPELINE_NAME = "mnist"

# wandb 설정
WANDB_API_KEY = "135017ad091dd0ca4896f8f5c427e9f982c2e9a9"

In [ ]:
ts = int(time.time())
BUCKET_NAME = f"gs://{BUCKET}"
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/{USER}"
WORKING_DIR = f"{PIPELINE_ROOT}/{ts}"
MODEL_DISPLAY_NAME = f"train_deploy{ts}"
MODEL_DIR = f"pipeline_root/{USER}/{ts}"

print(MODEL_DIR)
print(WORKING_DIR)
print(MODEL_DISPLAY_NAME)

vertex_ai.init(project=PROJECT_ID)

pipeline_root/JeongMin-Do/1632766559
gs://mlops-1234/pipeline_root/JeongMin-Do/1632766559
train_deploy1632766559


# Pipeline 정의

#### 해당 부분을 요구에 맞게 수정해주세요

In [ ]:
@component
def training_op(training_state: str):
    print("training task: {}".format(training_state))

@kfp.dsl.pipeline(name="resnet" + TIMESTAMP, pipeline_root=PIPELINE_ROOT)
def pipeline():
    train_task = training_op("model training")
    experimental.run_as_aiplatform_custom_job(
        train_task,
        worker_pool_specs=[
            {
                "containerSpec": {
                    "env": [{"name": "WANDB_API_KEY", "value": WANDB_API_KEY},
                            {"name": "WORKING_DIR", "value": WORKING_DIR}],
                    "imageUri": "silverstar456/vertexai:mnist", # 사전에 빌드한 도커 이미지 URI
                },
                "replicaCount": "1",
                "machineSpec": {
                    "machineType": "n1-standard-8", # 실행시 사용할 머신 타입 GCP 문서 참조
                    "accelerator_type": vertex_ai.gapic.AcceleratorType.NVIDIA_TESLA_V100, # GPU 타입
                    "accelerator_count": 1, # GPU 개수
                },
            }
        ],
    )

In [ ]:
compiler.Compiler().compile(
    pipeline_func = pipeline, package_path = "custom-pipeline.json"
)

pipeline = vertex_ai.PipelineJob(
    display_name="resnet",
    enable_caching=False,
    template_path="custom-pipeline.json",
    parameter_values={},
)

pipeline.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
